In [ ]:
!pip install PyPDF2


In [ ]:
import PyPDF2
import pandas as pd
import re

# Inicializar listas para cada coluna
datas = []
descricoes = []
valores = []
tipos = []

# Inicializar a variável `data`
data = None

# Abrir o PDF
with open("/content/df-digital_01012024_a_31012024_375025b8-1.pdf", "rb") as arquivo_pdf:
    leitor_pdf = PyPDF2.PdfReader(arquivo_pdf)

    # Percorrer cada página
    for num_pagina in range(len(leitor_pdf.pages)):
        pagina = leitor_pdf.pages[num_pagina]
        texto = pagina.extract_text()

        # Processar cada linha do texto da página
        for linha in texto.split('\n'):
            # Procurar datas
            data_match = re.match(r"(\d{2}/\d{2}/\d{4})", linha)
            if data_match:
                data = data_match.group(1)

            # Procurar transações
            transacao_match = re.search(r"([+-])\s*R\$\s*([\d.,]+)\s*(.*)", linha)
            if transacao_match and data is not None:
                tipo = 'Entrada' if transacao_match.group(1) == '+' else 'Saída'
                valor = float(transacao_match.group(2).replace('.', '').replace(',', '.'))
                descricao = transacao_match.group(3).strip()

                # Adicionar dados às listas
                datas.append(data)
                tipos.append(tipo)
                valores.append(valor)
                descricoes.append(descricao)

# Criar DataFrame com as listas
df = pd.DataFrame({
    'Data': datas,
    'Descrição': descricoes,
    'Valor': valores,
    'Tipo': tipos
})

# Visualizar DataFrame
print(df)


In [ ]:
import pandas as pd

# Suponha que você já tenha um DataFrame 'df' com todas as transações
# Criamos as funções de categorização como discutido antes

def categorizar_tipo_receita(descricao):
    descricao = descricao.lower()
    if "pix" in descricao:
        return "Pix"
    elif "boleto" in descricao:
        return "Boleto"
    elif "transferência" in descricao or "transf" in descricao:
        return "Transferência"
    elif "pagamento" in descricao:
        return "Pagamento"
    else:
        return "Outros"

def extrair_cliente_receita(descricao):
    palavras_chave = ["recebido", "recebida"]

    for palavra in palavras_chave:
        if palavra in descricao.lower():
            return descricao.split(palavra)[-1].strip()


def categorizar_tipo_despesa(descricao):
    descricao = descricao.lower()
    if "pix" in descricao:
        return "Pix"
    elif "boleto" in descricao:
        return "Boleto"
    elif "transferência" in descricao or "transf" in descricao:
        return "Transferência"
    elif "pagamento" in descricao:
        return "Pagamento"
    else:
        return "Outros"

def extrair_fornecedor_despesa(descricao):
    palavras_chave = ["pago", "pagamento", "Pix", "enviada", "conta"]

    for palavra in palavras_chave:
        if palavra in descricao:
           return descricao.split(palavra)[-1].strip()

# Filtrar e criar tabela de receitas
df_receitas = df[df['Tipo'] == 'Entrada'].copy()
df_receitas['Tipo de Receita'] = df_receitas['Descrição'].apply(categorizar_tipo_receita)
df_receitas['Cliente/Origem da Receita'] = df_receitas['Descrição'].apply(extrair_cliente_receita)
df_receitas = df_receitas[['Data', 'Valor', 'Tipo de Receita', 'Cliente/Origem da Receita']]

# Exibir as tabelas
print("Tabela de Receitas:")
print(df_receitas)

# Criando um arquivo CSV
df_receitas.to_csv("total_receitas.csv", index=False, encoding="utf-8")

print("Arquivo CSV gerado com sucesso!")



In [ ]:
# Filtrar e criar tabela de despesas
df_despesas = df[df['Tipo'] == 'Saída'].copy()
df_despesas['Tipo de Despesa'] = df_despesas['Descrição'].apply(categorizar_tipo_despesa)
df_despesas['Fornecedor/Destino da Despesa'] = df_despesas['Descrição'].apply(extrair_fornecedor_despesa)
df_despesas = df_despesas[['Data', 'Valor', 'Tipo de Despesa', 'Fornecedor/Destino da Despesa']]

print("\nTabela de Despesas:")
print(df_despesas)

# Criando um arquivo CSV
df_despesas.to_csv("total_despesas.csv", index=False, encoding="utf-8")

print("Arquivo CSV gerado com sucesso!")

In [ ]:
# Faturamento total
faturamento_total = df_receitas['Valor'].sum()
print(f"Faturamento Total: R${faturamento_total:.2f}")

In [ ]:
# Despesa

despesa_total = df_despesas['Valor'].sum()
print(f"Despesa Total: R${despesa_total:.2f}")

In [ ]:
#Lucro
lucro = faturamento_total - despesa_total
print(f"Lucro: R${lucro:.2f}")